<a href="https://colab.research.google.com/github/lenyabloko/SemEval2020/blob/master/SemEval2020_Paraphraser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

UPLOAD FILES - Place [train.csv](https://github.com/arielsho/Subtask-1/archive/master.zip) and [test.csv](https://github.com/arielsho/Subtask-1-test/archive/master.zip) files directly under your `gdrive/My Drive/Subtask-1/`, before starting (follow the prompt URL and get authentication token)

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

#!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-1/train.csv /content
#!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-1/test.csv /content

!cp -r /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-1/train-20180325-001253 /content
!cp -r /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-1/para-nmt-50m /content
!cp -r /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-1/paraphraser /content

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


START AGAIN

In [3]:
!pip install tensorflow
!pip install numpy

import os
os.environ['PYTHONPATH'] = "/usr/local/bin/python"
!echo $PYTHONPATH
!which python
!python --version

import sys
sys.path.append('/usr/local/lib/python3.6/site-packages/')

#%tensorflow_version 2.x
#import tensorflow
#print(tensorflow.__version__)

/usr/local/bin/python
/usr/local/bin/python
Python 3.6.9


In [0]:
# Restart runtime using 'Runtime' -> 'Restart runtime...'
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TEST INTERACTIVE

In [0]:
#from paraphraser import inference
#paraphrase = inference.greedy_paraphrase("Hello Woeldpa")
#print(paraphrase)

#!export LC_ALL=en_US.UTF-8
#!export LANG=en_US.UTF-8
#!sudo python -m spacy download en

!python -m spacy download en_core_web_sm
!python -m spacy download en

!python paraphraser/paraphraser/inference.py --checkpoint=/content/train-20180325-001253/model-171856

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')



2020-02-29 08:29:51.652439: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2020-02-29 08:29:51.654668: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x66f9340 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-02-29 08:29:51.654701: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-02-29 08:29:51.659283: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-02-29 08:29:51.845992: I tensorflow/strea

In [0]:
import tensorflow

import os
import sys
sys.path.append(os.getcwd()+'/paraphraser/paraphraser/')

import re

source_sentence="Folks might not have felt like paying $10 a ticket to see films that have proven to be rather disappointing at the box office, like Cinderella Man or Kingdomof Heaven, but it stands to reason -- as has always been the case -- that many people will probably rent them, or even buy them, on DVD."
source_sentence= re.sub(r"[,.;@#?!&]+\ *", " ", source_sentence)
print(source_sentence)

from inference import *

# for re-run - Restart runtime 
paraphraser = Paraphraser(checkpoint='/content/train-20180325-001253/model-171856')

paraphrases = paraphraser.sample_paraphrase(sentence=source_sentence, sampling_temp=0.75, how_many=3) 
for i, paraphrase in enumerate(paraphrases):
  print("Paraph #{}: {}".format(i, paraphrase))

FORMAT DATA

In [4]:
import pandas as pd

prefix = '/content/'
train_df = pd.read_csv(prefix + 'train.csv', header=None)
train_df=train_df.drop(index=0)

train_df = pd.DataFrame({
    'id':train_df[0],
    'labels':train_df[1],
    'alpha':['a']*train_df.shape[0],
    'text': train_df[2].replace(r'\n', ' ', regex=True)
})
train_df.head() 

,id,labels,alpha,text
1,100000,1,a,"Goodfellow's theory has been questioned, howev..."
2,100001,1,a,"However, both campaigners and pro-People's Vot..."
3,100002,1,a,Things could have been even better if the whol...
4,100003,0,a,"The new request, if approved, would keep the m..."
5,100004,0,a,Companies in financial difficulty can currentl...


In [8]:
import pandas as pd

count = train_df['id'].count()
print(count)

import os
import sys
sys.path.append(os.getcwd()+'/paraphraser/paraphraser/')

from inference import *
# if error next - Restart runtime to avoid re-initializing Paraphraser !!!
paraphraser = Paraphraser(checkpoint='/content/train-20180325-001253/model-171856')

import re
import time
from tqdm import tqdm

from inference import *

duplicate_df = train_df.copy()
paraphrased = []

for ind in tqdm(train_df.index):
  #print("Original: {}".format(train_df['text'][ind])) 
  original_sentence = train_df['text'][ind]

  source_sentence= re.sub(r"[~=+-\",:;#!&()0]+\ *", " ", original_sentence)
  paraphrases = paraphraser.sample_paraphrase(sentence=source_sentence, sampling_temp=0.75, how_many=3)

  for i, paraphrase in enumerate(paraphrases):
    duplicate = duplicate_df['id'] == train_df['id'][ind] # get identical row from duplicate dataframe
    duplicate['text'] = paraphrase
    #print("Paraph #{}: {}".format(i,duplicate['text']))
    #paraphrased.append(duplicate) 
  time.sleep(1)

print("Done")
train_df.append(paraphrased, ignore_index=True, sort=False) 
train_df.to_csv(prefix+'train.tsv', sep='\t', index=False, header=False)  

13000


ModuleNotFoundError: ignored

In [0]:
!cp /content/train.tsv /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-1/